In [ ]:
# regularize trajectories returns normalized trajectories as an array of shape
# (N,len(traj),dim+1) where, in addition to the various dimension, the time stamps are also given
def data_prepare_task3_inf(X,Y,dim):
    import numpy as np 
    thr=1e-10
    X=np.asarray(X)
    print(X.shape)
    N=len(X)
    #compute length of each trajectory
    trj_len=int(X.shape[1]/dim)
    xvec=np.zeros((N,trj_len-1,dim+1))
    for d in range(dim):
        x = np.array(X[:,d*trj_len:(d+1)*trj_len])
        x = np.diff(x,axis=1) 
        sx = np.std(x,axis=1)
        xvec[:,:,d] = (x-np.mean(x,axis=1).reshape(len(x),1)) / np.where(sx>thr,sx,1).reshape(len(x),1)   # normalize x data
    xvec[:,:,dim]=np.arange(trj_len-1)/trj_len
    # regularize labels
    label = []
    
    #label for changing point and alpha. entries from andi are (dimension, tc,class1,a1,class2,a2) 
#I want (a1,a2,sin(2pi*tc/T),cos(2pi*tc/T))
#NB it seems like the data shape chages when you produce or load the data!!! when you create it it also has the entry telling you the dimension
# if working with loaded data use corr=1 otherwise 0
    corr=0
    Y=np.asarray(Y)
    ss=np.shape(Y)
#print('shape is ',ss)
    ccl1=np.zeros((ss[0],4))
    ccl1[:,0]=Y[:,3-corr]
    ccl1[:,1]=Y[:,5-corr]
    ccl1[:,2]=np.sin((2*np.pi*Y[:,1-corr])/200)
    ccl1[:,3]=np.cos((2*np.pi*Y[:,1-corr])/200)

    
    
    label = ccl1
    return(xvec, label)

In [ ]:
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import losses, metrics

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import LSTM

from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.models import load_model
import scipy.io as sci

#dimension of system
dim=2


#Building the network
model_t2 = Sequential()
#first layer: LSTM of dimension 64
model_t2.add(LSTM(250,
                return_sequences=True,
                recurrent_dropout=0.2,
                input_shape=(None, dim+1)
                ))


model_t2.add(LSTM(50,
                    dropout=0,
                    recurrent_dropout=0.2,
                    ))


model_t2.add(Dense(4))



model_t2.compile(optimizer='adam',
                loss="mse",
                metrics=["mae"])

#Printing a summary of the built network
model_t2.summary()


In [ ]:
i = 200
N = 10
batch_sizes = [32, 128, 512, 2048]
dataset_used = [1, 4, 5, 20]
number_epochs = [5, 4, 3, 2]
n = 0
for batch in range(len(batch_sizes)):    
    for repeat in range(dataset_used[batch]):
        data = sci.loadmat('task3_1d' + str(i) + '_' + str(n) + '.mat')
        n += 1
        X = data['X'][0][0]
        #The label for the first model is in position 2 so
        Y = data['Y'][0][0].reshape(N,6)
        x, label = data_prepare_task3_inf(X,Y,dim)
        print(x.shape)
        history_t2 = model_t2.fit(x, 
                                label, 
                                epochs=number_epochs[batch], 
                                batch_size=batch_sizes[batch],
                                validation_split=0.1,
                                shuffle=True,
                                )

In [ ]:
batch_sizes = [512,]
dataset_used = [20,]
number_epochs = [2,]
n = 0
for batch in range(len(batch_sizes)):    
    for repeat in range(dataset_used[batch]):
        data = sci.loadmat('task3_1d' + str(i) + '_' + str(n) + '.mat')
        n += 1
        X = data['X'][0][0]
        Y = data['Y'][0][0].reshape(N,6)
        x, label = data_prepare_task3_inf(X,Y,dim)
        history_t2 = model_t2.fit(x, 
                                label, 
                                epochs=number_epochs[batch], 
                                batch_size=batch_sizes[batch],
                                validation_split=0.1,
                                shuffle=True,
                                )

In [ ]:
model_t2.save('Task3_'+str(dim)+'D_inference' + str(i) +'.h5')